In [1]:
import torch
import torch.nn as nn
from torch.nn import Embedding

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import dataset
import test_lstm

2023-03-10 22:05:30.267258: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
lie_trial_path = './processed_lie/' #60 entries
truth_trial_path = './processed_truth/' #61 entries
MU3D_path = './processed/' # 300 entries

In [3]:
# no split by person
numOfFrames = 10
X, Y = dataset.preprocessing(truth_trial_path, lie_trial_path, numOfFrames=numOfFrames)

TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [35]:
class positionalEncoder(nn.Module):

  def __init__(self, frame_length, encoding_length):
    super().__init__()

    embedding = nn.Embedding(frame_length, encoding_length)

    self.pe = embedding(torch.tensor([i for i in range(frame_length)]).unsqueeze(1)).squeeze()

  def forward(self, x):

    embedded = []
    for frame in x:
      embedded.append(torch.cat((frame, self.pe), 1).detach().numpy())

    return torch.tensor(embedded)

class classifierNetwork(nn.Module):

  def __init__(self, inFeatCount, num_T_layers, num_frames, num_fc_layers = 5, pos_encode_size = 5, n_heads = 5, n_hidden = 500, dropout = 0.3, outFeatCount = 2):
    super().__init__()

    self.posEncoder = positionalEncoder(num_frames, pos_encode_size)

    encoder_layer = nn.TransformerEncoderLayer(inFeatCount + pos_encode_size, n_heads, n_hidden, dropout)
    self.encoder = nn.TransformerEncoder(encoder_layer, num_T_layers)

    decoder_layer = nn.TransformerDecoderLayer(inFeatCount + pos_encode_size, n_heads)
    self.decoder = nn.TransformerDecoder(decoder_layer, num_T_layers)

    mid = ((inFeatCount + pos_encode_size) - outFeatCount) // 2 + outFeatCount

    self.fc1 = nn.Linear(inFeatCount + pos_encode_size, mid)
    self.fc2 = nn.Linear(mid, outFeatCount)

    self.init_weights()

  def init_weights(self):
      initrange = 0.1
      self.fc1.bias.data.zero_()
      self.fc1.weight.data.uniform_(-initrange, initrange)

      self.fc2.bias.data.zero_()
      self.fc2.weight.data.uniform_(-initrange, initrange)

  def forward(self, x):

    encoded = self.posEncoder(x)
    data = self.encoder(encoded)
    data = self.decoder(encoded, data)

    data = self.fc1(data)
    data = self.fc2(data)
    return nn.functional.softmax(data)      

In [6]:
pos_encoder_size = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
#generate embedded data
y_Train = []
y_Test = []

for i in range(yTrain.shape[0]):
    if yTrain[i] == 0:
        y_Train.append([[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0]])
    else:
        y_Train.append([[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]])

for i in range(yTest.shape[0]):
    if yTest[i] == 0:
        y_Test.append([[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0],[1,0]])
    else:
        y_Test.append([[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]])

xTrain = torch.tensor(xTrain).to(device).float()
xTest = torch.tensor(y_Train).to(device).float()

(49150, 10, 10)


In [36]:
featCount = 10
num_frames = 10
encoder_layers = 2

# Input shape = torch.tensor() of shape (num_frames, num_features)
# Output shape = list[lie prob, true prob] of shape (num_frames, 2)

net = classifierNetwork(featCount, encoder_layers, num_frames)

In [37]:
# training
import torch.optim as optim

def train(model, xTrain, yTrain, xTest, yTest, epochs = 100, lr = 0.001, batch_size = 10):
    """ Train a model on a dataset """

    loss_items = []
    accuracy_items = []
    test_accuracy_items = []
    
    # create a data loader to handle batching
    xTrain_loader =  torch.utils.data.DataLoader(xTrain, batch_size=batch_size, shuffle=False)
    xTest_loader = torch.utils.data.DataLoader(xTest, batch_size=batch_size, shuffle=False)

    # create a loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # train the model
    for epoch in range(epochs):

        # train

        idx = 0
        model.train()
        for batch in xTrain_loader:

            # get data
            x_train = batch.to(device).float()
            y_train = torch.tensor(yTrain[idx]).to(device)

            # forward pass
            y_pred = model(x_train)

            # compute loss
            loss = loss_fn(y_pred, y_train)

            # backward pass
            loss.backward()

            # update weights
            optimizer.step()
            optimizer.zero_grad()

        # evaluate
        model.eval()

        test_acc = 0
        acc = 0
        tot_loss = 0

        with torch.no_grad():
            
            idx_test = 0
            for batch in xTest_loader:
                xTest = batch.to(device).float()
                y_pred = model(xTest)
                y_test = torch.tensor(yTest[idx_test]).to(device)

                # compute test accuracy
                test_acc += (y_pred.argmax(0) == y_test).type(torch.float).mean().item()
                idx_test += 1
                

            test_acc /= len(xTest_loader)  
            test_accuracy_items.append(test_acc)

            idx_train = 0
            for batch in xTrain_loader:
                xTrain = batch.to(device).float()
                y_pred = model(xTrain)
                y_train = torch.tensor(yTrain[idx_train]).to(device)

                # compute accuracy
                acc += (y_pred.argmax(0) == y_train).type(torch.float).mean().item()
                idx_train += 1

                # computer loss
                tot_loss += loss_fn(y_pred, y_train).item()

            acc /= len(xTrain_loader)
            accuracy_items.append(acc)

        # store loss and accuracy
        loss_items.append(tot_loss)

        # print progress
        if (epoch+1) % 1 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {tot_loss:.4f}, Accuracy: {acc:.4f}, Test Accuracy: {test_acc:.4f}')

        idx += 1
            

train(net, xTrain, y_Train, xTest, y_Test)

/var/folders/66/pn8653k51_75x5ksbpsy8hm80000gn/T/ipykernel_23721/1381004600.py:54: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.functional.softmax(data)


Epoch 1/100, Loss: 11317.2071, Accuracy: 0.1044, Test Accuracy: 0.1022
Epoch 2/100, Loss: 11317.2071, Accuracy: 0.1052, Test Accuracy: 0.1044
Epoch 3/100, Loss: 11317.2071, Accuracy: 0.1056, Test Accuracy: 0.1037
Epoch 4/100, Loss: 11317.2071, Accuracy: 0.1048, Test Accuracy: 0.1025


KeyboardInterrupt: 